# Task 13 - Techniques for sampling parameter space

A parameter space may be surveyed for several reasons. In the context of tritium breeding, this may be to measure the impact of a parameter on tritium breeding.

There are many ways to sample a parameter space, but some provide significant advantages to others.

In [ ]:
# import dependencies for sampling and plotting

import argparse
import json
import math
import uuid
from pathlib import Path
import pandas as pd
import adaptive
import holoviews
import ipywidgets
import ghalton
import nest_asyncio
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
from plotly.subplots import make_subplots

adaptive.notebook_extension()
nest_asyncio.apply()

# imports pre-defined neutronics model and plotting tools
from openmc_model import find_tbr_hcpb
from plotting_tools import read_in_data, plot_simulation_results, plot_interpolated_results

# method for saving results in json file
def output_result(result):
    filename = "outputs/" + str(uuid.uuid4()) + ".json"
    Path(filename).parent.mkdir(parents=True, exist_ok=True)
    with open(filename, mode="w", encoding="utf-8") as f:
        json.dump(result, f, indent=4)
        
# method for showing results
def show_results(filtered_results_df):
    sample_trace = plot_simulation_results(filtered_results_df)
    fig = go.Figure()
    fig.add_trace(sample_trace)
    return fig

## Random Sampling

The easiest way to sample a parameter space is to use random sampling - where values are chosen at random from the parameter space.

The following code runs a neutronics simulation using a simple pre-defined model. The code randomly varies the input parameters of the model, runs the simulation and obtains the TBR. The parameters varied are the blanket enrichment and breeder percent in breeder plus multiplier ratio.

In [ ]:
# Define number of simulations

number_of_simulations = 16

In [ ]:
print("running simulations with random sampling")

for i in tqdm(range(number_of_simulations)):

    breeder_percent_in_breeder_plus_multiplier_ratio = np.random.uniform(0, 100)
    blanket_breeder_li6_enrichment = np.random.uniform(1, 100)

    result = find_tbr_hcpb(breeder_percent_in_breeder_plus_multiplier_ratio,
                           blanket_breeder_li6_enrichment)

    result["sample"] = "random"
    output_result(result)
    
results_df = pd.DataFrame(read_in_data())
filtered_results_df = results_df[results_df["sample"] == "random"]
show_results(filtered_results_df)

As shown, the simulations have been performed randomly across the parameter space.

The main advantage of random sampling is that it is an 'unbiased' sampling technique, meaning simulations are performed across the entire parameter space at the same rate. This means that all simulations contribute to the overall data trend and additional simulations can be easily performed to increase accuracy.

However, random sampling is an inefficient sampling technique as it leads to point clusters and voids across the parameter space. Some simulations are performed with very similar input parameters,but some areas of parameter space are sparsely sampled. This is inefficient because some simulations provide little additional information about the overall trend and their computational time would be better spent sampling sparsely-sampled areas.

Overall, random sampling is a simple technique for performing unbiased simulations over a parameter space, however, its poor spatial distribution of sample points makes it a highly inefficient technique.

## Grid Sampling

Another sampling technique is 'grid sampling', where samples are taken at regular intervals across the parameter space. This is an example of 'biased sampling' because the samples are ordered according to the grid specified.

The following code runs a neutronics simulation using a simple pre-defined model. The code varies the input parameters according to a specified grid. 
Li6 enrichment and breeder percent in breeder plus multiplier ratio are both varied between 0 and 100.

In [ ]:
print("running simulations with grid sampling")

number_of_steps = math.floor(math.sqrt(number_of_simulations))

for breeder_percent_in_breeder_plus_multiplier_ratio in tqdm(np.linspace(0, 100, number_of_steps)):
    for blanket_breeder_li6_enrichment in np.linspace(0, 100, number_of_steps):

        result = find_tbr_hcpb(breeder_percent_in_breeder_plus_multiplier_ratio,
                               blanket_breeder_li6_enrichment)

        result["sample"] = "grid"
        
        output_result(result)
        
results_df = pd.DataFrame(read_in_data())
filtered_results_df = results_df[results_df["sample"] == "grid"]
show_results(filtered_results_df)

Grid sampling maximises the spatial distribution of sample points across the parameter space by avoiding point clustering and, therefore, maximises the amount of useful information obtained from each simulation. As a result, grid sampling is a highly efficient sampling technique for covering a parameter space.

However, as simulations are performed in order according to the defined grid, they are biased towards the first parameter values in the grid, I.e. they are biased towards one area of parameter space. For example, the code abbove performs simulations with breeder percent in breeder plus multiplier ratio = 0 for all Li6 enrichments before breeder percent in breeder plus multiplier ratio is changed. This means that all simulations across the grid must be performed before a data trend across the entire parameter space can be observed. This is the main disadvantage of grid sampling as more simulations than are necessary may be performed, and it is difficult to efficiently add sample points to the existing data without performing a complete new grid search 'in-between' the existing data points. On the other hand, random sampling is unbiased meaning the data trend across the parameter space can be observed with a small number of simulations and additional sample points can be added easily.

Overall, grid sampling is a more efficient sampling technique than random sampling, but its bias towards initial parameter values makes it unsuitable for most applications. Instead, we tend to use more advanced 'quasi-random' or 'adaptive' techniques to improve sampling efficiency.

## Halton Sampling

Halton sampling is a quasi-random sampling technique based on the halton sequence. Using a quasi-random number sequence based on coprime numbers, halton sampling allows entire parameter spaces to be sampled efficiently and with an element of random nature. Like random sampling, halton sampling allows data trends across the entire parameter space to be observed with a small number of samples, but distributes the sample points more efficiently throughout the parameter space.

The following code runs a neutronics simulation using a simple pre-defined model. The code varies the input parameters according to the halton sequence. Li6 enrichment and breeder percent in breeder plus multiplier ratio are both varied between 0 and 100.

In [ ]:
print("running simulations with halton sampling")

sequencer = ghalton.Halton(2)
coords = sequencer.get(number_of_simulations)

for coord in tqdm(coords):

    breeder_percent_in_breeder_plus_multiplier_ratio = coord[0] * 100  # scales sampling from 0 to 100
    blanket_breeder_li6_enrichment = coord[1] * 100  # scales sampling from 0 to 100

    result = find_tbr_hcpb(breeder_percent_in_breeder_plus_multiplier_ratio,
                           blanket_breeder_li6_enrichment)

    result["sample"] = "halton"
    
    output_result(result)
    
results_df = pd.DataFrame(read_in_data())
filtered_results_df = results_df[results_df["sample"] == "halton"]
show_results(filtered_results_df)

As you can see, the sample points have a much better spatial distribution across the parameter shapce than random sampling.

The main advantage of halton sampling is that it allows the efficient sampling of an entire parameter space. Sample points are not clustered meaning each simulation provides a large amount of additional information about the overall data trend, and its quasi-random nature means additional samples can be added efficiently. However, the main disadvantage of halton sampling is that it still surveys the entire parameter space of interest. This can lead to the excessive sampling of 'flat' regions of the parameter space, i.e. areas where there is little variation in the overall trend, resulting in inefficiency.

Overall, halton sampling is better than random and grid sampling as it provides good spatial distribution and allows more samples to be added easily and efficiently. Ideally, however, we want to avoid over-sampling flat regions of the parameter space to further improve efficiency and reduce the number of simulations required. This is where 'adaptive' sampling is advantageous.

## Adaptive Sampling

Adaptive sampling is a sampling technique which uses data fitting to decide where in the parameter space to sample next. By fitting the data from samples that have already been taken, the overall data trend across the parameter space can be roughly predicted and an informed choice on where to sample the parameter space next can be made. Regions in the parameter space where the data trend is relatively flat do not have to be sampled as densely as rapidly changing regions. By allowing sample points to be chosen based on the data trend, computational time can be focused on the most important parts of the data trend.

The following code runs a neutronics simulation using a simple pre-defined model. Simulations begin by sampling the limits of the parameter space (i.e. (enrichment, breeder percentage) = (0, 100), (100, 0), (0, 100), (100, 100)) and then fitting these points to predict where TBR is varying most rapidly across the parameter space. A sample is then taken at this point and the process repeated. There are many ways to fit existing data points during adaptive sampling, however, this particular example uses gaussian process regression.

In [ ]:
print("running simulations with adaptive sampling")

def find_tbr(x):

    breeder_percent_in_breeder_plus_multiplier_ratio, blanket_breeder_li6_enrichment = x
                           
    result = find_tbr_hcpb(breeder_percent_in_breeder_plus_multiplier_ratio,
                           blanket_breeder_li6_enrichment)

    result["sample"] = "adaptive"
    
    output_result(result)

    return result["tbr"]

learner = adaptive.Learner2D(find_tbr, bounds=[(0, 100), (0, 100)])

runner = adaptive.Runner(learner, ntasks=1, goal=lambda l: l.npoints > number_of_simulations)

runner.live_info()

runner.ioloop.run_until_complete(runner.task)

results_df = pd.DataFrame(read_in_data())
filtered_results_df = results_df[results_df["sample"] == "adaptive"]
show_results(filtered_results_df)

As mentioned, the most important parts of a data trend are (usually) the regions where the data is changing as a function of parameter values. In our example, these are the regions where TBR is changing as a function of enrichment and breeder percentage. I.e. we do not want to excessively sample regions where TBR changes negligibly as a function of enrichment and breeder percentage. As shown, the parameter space is densely sampled in regions where TBR is changing most rapidly, and sparsely sampled in regions where TBR is changing negligibly.

The main advantage of adaptive sampling is that it is the most efficient technique for sampling a parameter space with an unknown distribution. By iteratively fitting the data and performing additional simulations we can determine an accurate distribution across the parameter space with fewer simulations than any other sampling technique. It is not a perfect solution, however, because over-sampling could still take place if we don't specify when to stop sampling. I.e. we would calculate the data fit and stop when we reach an acceptable uncertainty. Also could miss areas which have less prominent trends? I.e we don't get the whole picture across the whole parameter space.

Overall, adaptive sampling allows computational time to be focused on the most important parts of a distribution and is a highly efficient way of sampling a parameter space and, therefore, performing simulations.

To more accurately cover this parameter space more than the default 40 samples would be required.

## Plotting interpolated results

After sampling the parameter space of interest with a set of discrete simulations, we can interpolate between these results to deduce the data trend across the entire parameter space. Data can be fitted in several ways (e.g. linear, cubic, radial basis function, etc...) and contour plots can be created to show the overall data trend. The accuracy of the interpolated data is determined by the number of discrete data points meaning increasing the number of simulations performed increases the accuracy of the contour plot.

Run the code below to plot TBR interpolated across the parameter space.

In [ ]:
# Reading in and plotting results

results = read_in_data()
results_df = pd.DataFrame(results)

# arranging the 4 plots on one canvas
row_col_coords = [[1, 1], [1, 2], [2, 1], [2, 2], [3,1]]
sampling_methods = ["random", "grid", "halton", "adaptive"]

interpolated_fig = make_subplots(rows=2, cols=2, subplot_titles=(sampling_methods))

for sample, coords in zip(sampling_methods, row_col_coords):
    
    filtered_results_df = results_df[results_df["sample"] == sample]

    interpolated_traces = plot_interpolated_results(filtered_results_df)
    
    interpolated_fig.add_trace(interpolated_traces[0], row=coords[0], col=coords[1])
    interpolated_fig.add_trace(interpolated_traces[1], row=coords[0], col=coords[1])
    
interpolated_fig.update_xaxes(title_text="Li6 enrichment percent")
interpolated_fig.update_yaxes(title_text="Breeder percent in breeder plus multiplier volume")
interpolated_fig.show()

The graphs show how the choice of sampling technique has a large impact on the interpolated data. Note how grid, halton and random sampling result in a large number of simulations being performed in regions where variation in TBR is small (large Li6 enrichment and breeder percent) meaning simulations are wasted. On the other hand, adaptive sampling avoids this region, meaning the number of wasted simulations is minimised.

**Learning Outcomes for Task 13:**

- The optimal breeder percent in breeder plus multiplier volume changes for different amounts of lithium enrichment.
- Increasing lithium 6 enrichment tends to increase the TBR.
- Random selection of parameters is not an efficient way of covering the search space.
